## After the initial breadth first search, we take the list of scraped links and create a dictionary. This will go back to the openwpm scraper to get HTML

In [1]:
import pickle
import pandas as pd
from pathlib import Path
import os
from preprocessing_functions import *

# # REAL SITES
# with open('500_real_list.pkl', 'rb') as f:
    # src_list_real = pickle.load(f)
# f.close()
# outfile_real = '500_real_df.pkl'
# path_to_src_lists_real = Path.cwd().parent.parent / 'datadir' / 'newsguard' / 'real_links' / 'sources'

# failed_real, unsaved_real = check_for_unscraped_sites(path_to_src_lists_real, src_list_real)

# # FAKE SITES
# with open('500_fake_list.pkl', 'rb') as f:
    # src_list_fake = pickle.load(f)
# f.close()
# outfile_fake = '500_fake_df.pkl'
# path_to_src_lists_fake = Path.cwd().parent.parent / 'datadir' / 'newsguard' / 'fake_links' / 'sources'


# failed_fake, unsaved_fake = check_for_unscraped_sites(path_to_src_lists_fake, src_list_fake)

# REAL SITES
with open('GDI_list.pkl', 'rb') as f:
    gdi_list = pickle.load(f)
f.close()
outfile_real = 'GDI_df.pkl'
path_to_src_lists = Path.cwd().parent.parent / 'datadir' / 'GDI' / 'links' / 'sources'

failed_gdi, unsaved_gdi = check_for_unscraped_sites(path_to_src_lists, gdi_list)
evaluate_failed_sites(failed_gdi, unsaved_gdi)

FAKE FAILED
[2, 8, 18, 21, 39, 41, 61, 76, 122, 141, 148, 170, 186, 190, 191, 192, 199, 212, 230, 243, 246, 248, 249, 271, 276, 279, 333, 345, 371, 379, 387, 391, 418, 423, 434, 449, 462, 474, 476, 487, 500, 517, 527, 543, 549, 551, 552, 572, 605, 610, 645, 667, 699, 703, 714, 728, 731, 752, 776, 778, 781, 782, 783, 799, 829, 831, 857, 883, 890, 907, 909, 913, 918, 950, 954, 955, 956, 968, 969, 1002, 1031, 1036, 1090, 1100, 1107, 1108, 1126, 1128, 1136, 1171, 1176, 1186, 1188, 1214, 1215, 1222, 1223, 1224, 1245, 1257]
['http://ahtribune.com', 'http://alphanewsmn.com', 'http://americanfreedombybarbara.com', 'http://americanmarxism.net', 'http://anguillesousroche.com', 'http://anonews.co', 'http://babybiden.com', 'http://biggovernment.com', 'http://cartoonstock.com', 'http://chicksontheright.com', 'http://christophercantwell.com', 'http://collective-evolution.com', 'http://conservativefighters.com', 'http://conservativehq.com', 'http://conservativemedia.com', 'http://conservativeminute.c

## When lots of sites are failed and unsaved, modify the CSV so that new sites can be put in their place

In [3]:
# Load CSVs

real_csv = pd.read_csv(Path.cwd().parent / 'fake_real_CSVS' / '500_real.csv')
fake_csv = pd.read_csv(Path.cwd().parent / 'fake_real_CSVS' / '500_fake.csv')

# Change these with each rescrape
unsaved_col_name = 'unsaved_3'
failed_col_name = 'failed_3'

def modify_csv_for_failed_sites(df, unsaved, failed, unsaved_col_name, failed_col_name):
    '''
    Takes df (read from a csv) and modifies it so that new sites can be manually added after a scrape fails

    ~~~~ ARGUMENTS ~~~~
    df : DataFrame
        - read from csv, for example 500_real.csv
    unsaved : dictionary
        - Output of check_for_unscraped_sites()
    failed: dictionary
        - Output of check_for_unscraped_sites()

    ~~~~ RETURNS ~~~~
    modified_df : DataFrame
        - Can be saved as a CSV, formatted for easier manual site addition
    '''
    # Insert columns if they're not there already
    if failed_col_name not in df.columns:
        df[failed_col_name] = ''
    if unsaved_col_name not in df.columns:
        df[unsaved_col_name] = ''
    
    for k,v in unsaved.items():
        df.at[k, unsaved_col_name] = df.loc[[k]]['source'].values[0]
        df.at[k, 'source'] = ''
    for k,v in failed.items():
        df.at[k, failed_col_name] = df.loc[[k]]['source'].values[0]
        df.at[k, 'source'] = ''
    
    return df

real_csv_modified = modify_csv_for_failed_sites(real_csv, unsaved_real, failed_real, unsaved_col_name, failed_col_name)
fake_csv_modified = modify_csv_for_failed_sites(fake_csv, unsaved_fake, failed_fake, unsaved_col_name, failed_col_name)

# Save new csvs
fake_csv_modified.to_csv('500_fake.csv', index=False)
real_csv_modified.to_csv('500_real.csv', index=False)

## After the CSV is modified, generate a new custom list of sites to scrape.
this can be implemented in bfls_<fake or real>.py as new_idx and new_list. Which should be accessed in the command loop

In [4]:
fake_csv = pd.read_csv(Path.cwd().parent / 'fake_real_CSVs' / '500_fake.csv')
real_csv = pd.read_csv(Path.cwd().parent / 'fake_real_CSVs' / '500_real.csv')

def get_rescrape_lists(df, unsaved_col_name, failed_col_name):
    '''
    Given a dataframe read from a CSV with new sites to scrape, this function will generate lists to put in the scraping script
    '''
    failed_idx = df[failed_col_name].notnull().values
    unsaved_idx =  df[unsaved_col_name].notnull().values
    f_idx = [i for i, x in enumerate(failed_idx) if x]
    u_idx = [i for i, x in enumerate(unsaved_idx) if x]
    
    modified_idx = f_idx + u_idx
    modified_idx.sort()

    urls = []
    # Get new list of urls
    for idx in modified_idx:
        urls.append(df.at[idx, 'source'])

    urls = ['http://' + url for url in urls]
 
    return modified_idx, urls

rescrape_indices_fake, rescrape_urls_fake = get_rescrape_lists(fake_csv, unsaved_col_name, failed_col_name)
print('FAKE RESCRAPE')
print(rescrape_indices_fake)
print(rescrape_urls_fake)

rescrape_indices_real, rescrape_urls_real = get_rescrape_lists(real_csv, unsaved_col_name, failed_col_name)
print('REAL RESCRAPE')
print(rescrape_indices_real)
print(rescrape_urls_real)


FAKE RESCRAPE
[352]
['http://thenorthstar.com']
REAL RESCRAPE
[]
[]


In [2]:
pd.set_option('display.max_rows', 100)



df_real = generate_link_dataframe(path_to_src_lists_real, src_list_real, outfile_real)
df_fake = generate_link_dataframe(path_to_src_lists_fake, src_list_fake, outfile_fake)

In [2]:
# For GDI

df_GDI = generate_link_dataframe(path_to_src_lists, gdi_list, outfile_real, failed_sites=failed_gdi, unsaved_sites=unsaved_gdi)

In [3]:
df_GDI.head()

,site,num_links,link_dict
index,,,
0,http://ae911truth.org,3605,"{0: 'https://www.ae911truth.org/', 1: 'https:/..."
1,http://afa.net,108,"{0: 'https://afa.net/', 1: 'https://afa.net/th..."
3,http://aim.org,522,"{0: 'https://www.aim.org/', 1: 'https://www.ai..."
4,http://aim4truth.org,97,"{0: 'https://aim4truth.org/', 1: 'https://aim4..."
5,http://alipac.us,166,"{0: 'http://www.alipac.us/content/', 1: 'https..."
